## Question 1
#### Multiple Knapsack Problem

In [5]:
import gurobipy as gp
from gurobipy import *
import numpy as np
import pandas as pd
import time as time

In [6]:
rnd = np.random

#### specifying the number of items 

In [7]:
N = 500
K = 30

(NB: using large numbers does not work for this gurobi academic version )

In [8]:
n=[i for i in range(1, N + 1)]
k=[j for j in range(1, K + 1)]

#### randomly generating the value (v), weight (w), and capacity (c) of these knapsacks

In [9]:
v = {i: rnd.randint(10, 100) for i in n}
w = {i: rnd.randint(10, 1000) for i in n}
c = {j: rnd.randint(100, 3000) for j in k} # we need K capacity for k knapsacks

#### Model Initialization

In [10]:
model = gp.Model('Knapsack')

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-26


#### our decision variable

In [11]:
x={}
for i in n:
    for j in k:
        x[i,j] = model.addVar(vtype=GRB.BINARY, name="x(%s,%s)" % (i,j))

#### item constraint

In [12]:
for i in n:
    model.addConstr(quicksum(x[i,j] for j in k)<=1)

#### weight constraint

In [13]:
for j in k:
    model.addConstr(quicksum(w[i] * x[i, j] for i in n ) <= c[j])

#### objective function

In [14]:
model.setObjective(quicksum(x[i, j]*v[i] for i in n for j in k), GRB.MAXIMIZE)

#### updating and writing the model

In [15]:
model.update()
model.write('0knapsack.lp')

#### solving the model and printing the solution

In [ ]:
start = time.time()
model.optimize()
end = time.time()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i3-1000NG4 CPU @ 1.10GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 530 rows, 15000 columns and 30000 nonzeros
Model fingerprint: 0xaa66ca71
Variable types: 0 continuous, 15000 integer (15000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+03]
Found heuristic solution: objective 6041.0000000
Presolve removed 0 rows and 2664 columns
Presolve time: 0.12s
Presolved: 530 rows, 12336 columns, 24672 nonzeros
Variable types: 0 continuous, 12336 integer (12336 binary)
Found heuristic solution: objective 8460.0000000

Root relaxation: objective 1.183264e+04, 1323 iterations, 0.08 seconds (0.08 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   

In [ ]:
def print_solution(model):
    if model.status != GRB.OPTIMAL:
        print("Model is not Optimized")
    
    else:
        print('Objective function value: ', model.objVal)
        print('Solution time: ', end - start, ' seconds')
        print('=================================================')
        vars = model.getVars()
        values = model.getAttr('x', vars)
        for var, val in zip(vars, values):
            if val > 1e-6:
                print(f"{var.varName}: {val}")

In [ ]:
print_solution(model)

## QUESTION 2 

Reading the data from "Transportation.xlsx"

In [ ]:
A = pd.read_excel("/Users/mawulitsimese/Library/Containers/com.microsoft.Excel/Data/Desktop/Transportation.xlsx")

In [ ]:
A = A.fillna(0)

In [ ]:
A

#### taking out capacity column and demand row

In [ ]:
B = A.iloc[:5, :9]
B

In [ ]:
origin = B['City'].tolist()
capacity = A.iloc[:5,9:10].values.flatten().tolist()

In [ ]:
dest = list(B.columns.values)
dest.remove('City')
demand = A.iloc[5:6,1:9].values.flatten().tolist()

In [ ]:
originloc = [m for m in origin]
destloc = [n for n in dest]
assign = [(p, j) for p in origin for j in dest]


In [ ]:
arr = B.iloc[0:, 1:].to_numpy().flatten()

In [ ]:
Dict= {assign[i]: arr[i] for i in range(len(arr))}

In [ ]:
assignments, costs = gp.multidict(Dict)

#### making sure the demand dont exceed capacity

In [ ]:
Supply= {originloc[i]: capacity[i] for i in range(len(capacity))}
Demand= {destloc[j]: demand[j] for j in range(len(demand))}

In [ ]:
if sum(Supply.values()) < sum(Demand.values()):
    diff = sum(Demand.values()) - sum(Supply.values()) 
    Supply[m] += diff

#### now we write the model

In [ ]:
model = gp.Model('Transportation')

In [ ]:
x={}
for i in origin:
    for j in dest:
        x[i,j] = model.addVar(vtype="C", name="x(%s,%s)" % (i,j))

In [ ]:
for i in origin:
    model.addConstr(quicksum(x[i, j] for j in dest ) <= Supply[i] , name="Supply(%s)" %i)

In [ ]:
for j in dest:
    model.addConstr(quicksum(x[i, j] for i in origin ) >= Demand[j] , name="Demand(%s)" % j)

In [ ]:
model.setObjective(quicksum(x[i, j]*Dict[i, j] for i, j in assign), GRB.MINIMIZE)

In [ ]:
model.update()
model.write('0transport.lp')

In [ ]:
start = time.time()
model.optimize()
end = time.time()

In [ ]:
def print_solution(model):
    if model.status != GRB.OPTIMAL:
        print("Model is not Optimized")
    
    else:
        print('Objective function value: ', model.objVal)
        print('Solution time: ', end - start, ' seconds')
        print('=================================================')
        vars = model.getVars()
        values = model.getAttr('x', vars)
        for var, val in zip(vars, values):
            if val > 1e-6:
                print(f"{var.varName}: {val}") 

In [ ]:
print_solution(model)